## Assignment 1

### Task 1: Reading in the Data

In [1]:
import numpy as np
import pandas as pd

In [2]:
event_def = pd.read_csv("data/Event Definitions.csv")

In [3]:
event_def.sort_values(by='event_definition_id')

,event_name,journey_id,event_definition_id,milestone_number,stage
10,campaign_click,1,2,NaN,Discover
6,application_web_submit,1,3,NaN,Apply for Credit
21,browse_products,1,4,NaN,First Purchase
24,view_cart,1,5,NaN,First Purchase
20,begin_checkout,1,6,NaN,First Purchase
23,place_order_web,1,7,2.0,First Purchase
17,place_downpayment,1,8,4.0,Downpayment
11,customer_requested_catalog_digital,1,9,NaN,Discover
12,fingerhut_university,1,10,NaN,Discover
19,add_to_cart,1,11,NaN,First Purchase


In [4]:
df = pd.read_csv(
    "data/dat_train1.csv",
    low_memory=True,
    parse_dates=['event_timestamp'],
    usecols=['customer_id', 'account_id', 'ed_id', 'event_timestamp', 'journey_steps_until_end'],
    dtype={
        'customer_id': np.int32,
        'account_id': np.int32,
        'ed_id': np.uint8,
        'journey_steps_until_end': np.uint16
    }
)

In [5]:
df.dtypes

customer_id                              int32
account_id                               int32
ed_id                                    uint8
event_timestamp            datetime64[ns, UTC]
journey_steps_until_end                 uint16
dtype: object

In [6]:
df.head()

,customer_id,account_id,ed_id,event_timestamp,journey_steps_until_end
0,15849251,383997507,4,2021-11-04 14:11:15+00:00,1
1,15849251,383997507,4,2021-11-04 14:11:29+00:00,2
2,15849251,383997507,4,2021-11-04 14:12:10+00:00,3
3,15849251,383997507,4,2021-11-04 14:12:21+00:00,4
4,15849251,383997507,4,2021-11-04 14:12:24+00:00,5


In [7]:
df.shape

(54960961, 5)

In [13]:
len(df[['customer_id', 'account_id']].value_counts())

1430445

In [14]:
df['event_timestamp'].min()

Timestamp('2020-11-03 03:31:30+0000', tz='UTC')

In [15]:
df['event_timestamp'].max()

Timestamp('2023-01-23 12:29:56+0000', tz='UTC')

1. The dataset has 54,960,961 rows.
2. There are 1,430,445 unique Ids in the dataset.
3. The earliest time stamp is `Timestamp('2020-11-03 03:31:30+0000', tz='UTC')` and the latest time stamp is `Timestamp('2023-01-23 12:29:56+0000', tz='UTC')`.

### Task 2: Remove Duplicates

In [17]:
num_duplicates = df.duplicated(
    subset=['customer_id', 'account_id', 'ed_id', 'event_timestamp']
).sum()
print(f"There are {num_duplicates} duplicate rows, which is {num_duplicates / df.shape[0]} of all rows.")

There are 3112100 duplicate rows, which is 0.056623827956720045 of all rows.


In [19]:
df.drop_duplicates(
    subset=['customer_id', 'account_id', 'ed_id', 'event_timestamp'],
    inplace=True
)

In [20]:
print(f"There are {df.shape[0]} rows after removing duplicates.")

There are 51848861 rows after removing duplicates.


1. 3,112,100 entries are duplicates, which makes up about 0.0566 of all rows.
2. After deleting duplicates, there are 51,848,861 in the data set.

Now let's recompute the `journey_steps_until_end` column, ensuring that the ordering of events within each journey is correct.

Each journey can be uniquely identified by the combination of `customer_id` and `account_id`.

In [38]:
df.sort_values(by=['customer_id', 'account_id', 'event_timestamp'], inplace=True)

In [46]:
df['journey_steps_until_end'] = df.groupby(['customer_id', 'account_id']).cumcount() + 1

In [48]:
df.reset_index(drop=True, inplace=True)

In [55]:
# Save the final data frame
df.to_csv("data/cleaned_dat_train1.csv", index=False)